In [1]:
!python -m pip install --upgrade pip
!pip install pandas
!pip install gspread
!pip install numpy
!pip install matplotlib

  Using cached pip-23.3.1-py3-none-any.whl (2.1 MB)
  Attempting uninstall: pip
    Found existing installation: pip 22.3.1
    Uninstalling pip-22.3.1:


ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'c:\\python311\\lib\\site-packages\\pip-22.3.1.dist-info\\entry_points.txt'
Consider using the `--user` option or check the permissions.


[notice] A new release of pip available: 22.3.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.3.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.3.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.3.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.3.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [64]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import warnings
import gspread
warnings.filterwarnings('ignore')

In [65]:
gc = gspread.service_account(filename = "C:\Data Science\KNN\Knn_gender_height_weight\linear-regression-404916-8d9c449085dc.json")

In [66]:
sheet = gc.open_by_url("https://docs.google.com/spreadsheets/d/10Rn9DxaOxs_1oiQwymWyjf47nc6d2vLCam2l6stDQgY/edit?usp=sharing")

In [67]:
ws = sheet.worksheet('weight-height')

In [68]:
df = pd.DataFrame(ws.get_all_records())

In [69]:
df

,Gender,Height,Weight
0,Male,73.847017,241.893563
1,Male,68.781904,162.310473
2,Male,74.110105,212.740856
3,Male,71.730978,220.042470
4,Male,69.881796,206.349801
...,...,...,...
8550,Female,60.483946,110.565497
8551,Female,63.423372,129.921671
8552,Female,65.584057,155.942671
8553,Female,67.429971,151.678405


In [70]:
df['Gender'].value_counts()

Gender
Male      5000
Female    3555
Name: count, dtype: int64

In [71]:
df.isnull().sum()

Gender    0
Height    0
Weight    0
dtype: int64

In [72]:

from sklearn.preprocessing import OneHotEncoder

In [73]:
ohe = OneHotEncoder(sparse=False)

In [74]:
gender_ohe = ohe.fit_transform(df[['Gender']])

In [75]:
gender_ohe

array([[0., 1.],
       [0., 1.],
       [0., 1.],
       ...,
       [1., 0.],
       [1., 0.],
       [1., 0.]])

In [76]:
df.drop(columns='Gender', inplace=True)

In [77]:
df.values

array([[ 73.84701702, 241.8935632 ],
       [ 68.78190405, 162.3104725 ],
       [ 74.11010539, 212.7408556 ],
       ...,
       [ 65.58405676, 155.9426708 ],
       [ 67.42997138, 151.6784055 ],
       [ 60.92179087, 131.2537378 ]])

In [78]:
combined_values = np.hstack((gender_ohe,df[['Height', 'Weight']].values))

In [79]:
combined_values

array([[  0.        ,   1.        ,  73.84701702, 241.8935632 ],
       [  0.        ,   1.        ,  68.78190405, 162.3104725 ],
       [  0.        ,   1.        ,  74.11010539, 212.7408556 ],
       ...,
       [  1.        ,   0.        ,  65.58405676, 155.9426708 ],
       [  1.        ,   0.        ,  67.42997138, 151.6784055 ],
       [  1.        ,   0.        ,  60.92179087, 131.2537378 ]])

In [80]:
encoded_column_names = ohe.get_feature_names_out(['Gender'])
columns = list(encoded_column_names) + ['Height', 'Weight']

In [81]:
combined_df = pd.DataFrame(combined_values, columns=columns)

In [82]:
combined_df

,Gender_Female,Gender_Male,Height,Weight
0,0.0,1.0,73.847017,241.893563
1,0.0,1.0,68.781904,162.310473
2,0.0,1.0,74.110105,212.740856
3,0.0,1.0,71.730978,220.042470
4,0.0,1.0,69.881796,206.349801
...,...,...,...,...
8550,1.0,0.0,60.483946,110.565497
8551,1.0,0.0,63.423372,129.921671
8552,1.0,0.0,65.584057,155.942671
8553,1.0,0.0,67.429971,151.678405


In [86]:
from sklearn.model_selection import train_test_split

In [87]:
x = combined_df.drop('Weight', axis=1)
y = combined_df[['Weight']]

In [88]:
x.head()

,Gender_Female,Gender_Male,Height
0,0.0,1.0,73.847017
1,0.0,1.0,68.781904
2,0.0,1.0,74.110105
3,0.0,1.0,71.730978
4,0.0,1.0,69.881796


In [89]:
y.head()

,Weight
0,241.893563
1,162.310473
2,212.740856
3,220.042470
4,206.349801


In [90]:
xtrain, xtest, ytrain, ytest = train_test_split(x,y, test_size=0.3)

In [91]:
xtrain.head()

,Gender_Female,Gender_Male,Height
6367,1.0,0.0,58.387272
3354,0.0,1.0,68.087904
7718,1.0,0.0,69.125275
4708,0.0,1.0,68.066867
6408,1.0,0.0,63.168648


In [92]:
xtest.head()

,Gender_Female,Gender_Male,Height
3050,0.0,1.0,69.191824
1734,0.0,1.0,65.819978
8451,1.0,0.0,62.015811
6978,1.0,0.0,61.116609
6191,1.0,0.0,64.110499


In [93]:
ytrain.head()

,Weight
6367,85.793085
3354,170.125245
7718,165.830008
4708,187.038345
6408,142.010379


# Linear Regression

In [56]:
from sklearn.linear_model import LinearRegression 

In [57]:
reg = LinearRegression()

In [94]:
reg.fit(xtrain, ytrain) 

LinearRegression()

In [95]:
reg.predict(xtest) 

array([[187.78125   ],
       [167.69140625],
       [125.78515625],
       ...,
       [157.7109375 ],
       [124.2578125 ],
       [142.6484375 ]])

In [96]:
ytest['predicted_y'] = reg.predict(xtest)

In [97]:
ytest.head()

,Weight,predicted_y
3050,166.253276,187.781250
1734,172.746365,167.691406
8451,133.652830,125.785156
6978,101.097500,120.429688
6191,146.756263,138.265625


In [98]:
ytest.drop('predicted_y', axis=1, inplace=True)

In [99]:
ytest.head()

,Weight
3050,166.253276
1734,172.746365
8451,133.652830
6978,101.097500
6191,146.756263


In [100]:
reg.score(xtrain, ytrain)

0.8971369498650855

In [101]:
reg.score(xtest, ytest)

0.9061056977079547

In [102]:
from sklearn.metrics import mean_squared_error

In [103]:
mse = mean_squared_error(ytest, reg.predict(xtest))

In [104]:
mse

98.71922428814614

# KNN Regressor

In [137]:
from sklearn.neighbors import KNeighborsRegressor

In [148]:
neigh = KNeighborsRegressor(n_neighbors=3)

In [149]:
neigh.fit(xtrain, ytrain) 

KNeighborsRegressor(n_neighbors=3)

In [150]:
neigh.predict(xtest)

array([[189.8093608 ],
       [160.94447783],
       [126.65411977],
       ...,
       [152.6339523 ],
       [124.62464067],
       [149.03368647]])

In [151]:
ytest['predicted_y_by_regression'] = reg.predict(xtest)

In [152]:
ytest['predicted_y_by_KNN']= neigh.predict(xtest)

In [153]:
ytest

,Weight,predicted_y_by_regression,predicted_y_by_KNN
3050,166.253276,187.781250,189.809361
1734,172.746365,167.691406,160.944478
8451,133.652830,125.785156,126.654120
6978,101.097500,120.429688,115.484532
6191,146.756263,138.265625,137.611746
...,...,...,...
8317,155.888981,153.179688,155.480620
6050,125.601269,110.785156,112.788872
3102,159.633740,157.710938,152.633952
6426,110.025793,124.257812,124.624641


In [154]:
neigh.score(xtrain, ytrain)

0.9294817130756148

In [155]:
ytest.drop('predicted_y_by_regression', axis=1, inplace=True)

In [156]:
ytest.drop('predicted_y_by_KNN', axis=1, inplace=True)

In [157]:
neigh.score(xtest, ytest)

0.8729833845750263

In [158]:
mse = mean_squared_error(ytest, neigh.predict(xtest))

In [159]:
mse

133.54358507780796